# 最优潮流分析：第二周学习笔记
本周的核心任务聚焦在最优潮流OPF问题的理解与编程实现，下面是本周项目的详细笔记
---

## 1. 电力系统导纳矩阵 Ybus 构建

### 理论知识：
- 电力网络由母线与支路组成，可用导纳矩阵 Ybus 表示其拓扑与电气特性。
- Ybus 是对称复数矩阵，反映节点之间导纳关系。
- 公式构建：
  $$
  Y_{ii} = \sum_{j \ne i} y_{ij}, \quad Y_{ij} = -y_{ij}
  $$

### 示例代码：



In [15]:
import json
import numpy as np

def load_case(filepath):
    with open(filepath, 'r') as f:
        return json.load(f)

def build_Ybus(case):
    n = len(case["buses"])
    Y = np.zeros((n, n), dtype=complex)
    for br in case["branches"]:
        i, j = br["from"]-1, br["to"]-1
        y = 1 / complex(br["r"], br["x"])
        Y[i, i] += y
        Y[j, j] += y
        Y[i, j] -= y
        Y[j, i] -= y
    return Y

## 2. 功率不平衡量计算（power_mismatch.py）

#### 不平衡量定义：
  - 我们希望系统中每个节点满足功率平衡： $𝑃_{注入} = 𝑃_{需求}$ ,$𝑄_{注入} = 𝑄_{需求}$，而一开始我们只能猜电压（初始值），算出来的功率不一定刚好满足需求。
  于是我们定义不平衡量：

$$
\Delta P_i = 𝑃_i^{指定} − 𝑃_𝑖^{计算}, \Delta Q_i = Q_i^{指定} − Q_𝑖^{计算}
$$


  - 如果 $\Delta P$、$\Delta Q$ 接近 0，说明系统满足功率平衡；如果不为 0，就说明当前电压估计不对，需要修正。

- 若 $\Delta P_i$ 和 $\Delta Q_i$ 非零，表示电网尚未达到平衡状态；
- 节点功率：$$S_i = P_i + jQ_i = V_i \sum_{j=1}^n V_j^* Y_{ij}^*$$
- 功率不平衡量计算是牛顿法迭代的核心。

### 代码实现：

In [16]:
def calculate_power_mismatch(Ybus, V, theta, P_spec, Q_spec, bus_types):
    n = len(V)
    P_calc = np.zeros(n)
    Q_calc = np.zeros(n)
    for i in range(n):
        for j in range(n):
            Y = Ybus[i, j]
            Vi, Vj = V[i], V[j]
            angle = theta[i] - theta[j] - np.angle(Y)
            P_calc[i] += Vi * Vj * abs(Y) * np.cos(angle)
            Q_calc[i] -= Vi * Vj * abs(Y) * np.sin(angle)
    dP = P_spec - P_calc
    dQ = Q_spec - Q_calc
    return dP, dQ


## 3. 牛顿-拉夫逊潮流求解器（newton_solver.py）
### 理论知识:
-  $J$ 构建线性化模型：
$$
\Delta X = -j^{-1}F(X)
$$
-  逐步迭代修正 $|V|$ 和 $\theta$ 直到误差收敛；

- 常见收敛标准：$\max(|\Delta P|, |\Delta Q|) < 10^{-6}$；

- 最终输出每个母线的 $|V|$ 和 $\theta$。

### 代码实现：

In [17]:
def solve_powerflow(Ybus, P_spec, Q_spec, V0, theta0, bus_types, tol=1e-6):
    V, theta = V0.copy(), theta0.copy()
    for iteration in range(20):
        dP, dQ = calculate_power_mismatch(Ybus, V, theta, P_spec, Q_spec, bus_types)
        J = build_jacobian(Ybus, V, theta, bus_types)
        mismatch = np.concatenate((dP, dQ))
        if np.max(np.abs(mismatch)) < tol:
            break
        delta = np.linalg.solve(J, mismatch)
        theta += delta[:len(theta)]
        V += delta[len(theta):]
    return V, theta


## 4. 潮流结果可视化与结构重构（visualize_powerflow.py）
### 理论知识：
- 电网结果可通过图形或表格方式展示电压分布；
- 为 OPF 构建接口（如生成带发电机参数的 json）；
- 支持格式转换如 JSON → numpy → Gurobi。
### 实现过程：
- 电压幅值可视化：plot_voltage_magnitude()：
- 结构重构与生成 OPF 数据：
    - 通过读取已有网络结构文件3bus.json，添加发电机信息（如出力上下限、成本），并且保存为新的 JSON 文件用于构造 OPF 模型的输入数据。



In [ ]:
def add_generators_to_case(input_file, output_file, generators):
    """
    向已有的电网数据中添加发电机信息，并保存为新的 JSON 文件
    参数:
        input_file: 原始 JSON 文件路径
        output_file: 修改后输出文件路径
        generators: 发电机信息列表，如：
            [{"bus": 1, "pmin": 0, "pmax": 6, "cost": 10}]
    """
    with open(input_file, 'r') as f:
        case = json.load(f)

    case["generators"] = generators

    with open(output_file, 'w') as f:
        json.dump(case, f, indent=2)

    print(f"已生成带发电机的数据文件：{output_file}")

### JSON文件结构示例：

In [18]:
{
  "buses": [{"id": 1, "type": "PV"}, {"id": 2, "type": "PQ"}],
  "branches": [{"from": 1, "to": 2, "r": 0.0, "x": 0.1}],
  "generators": [{"bus": 1, "pmin": 0, "pmax": 6, "cost": 10}]
}

{'buses': [{'id': 1, 'type': 'PV'}, {'id': 2, 'type': 'PQ'}],
 'branches': [{'from': 1, 'to': 2, 'r': 0.0, 'x': 0.1}],
 'generators': [{'bus': 1, 'pmin': 0, 'pmax': 6, 'cost': 10}]}

## 5. 最优潮流线性建模（opf_linear.py）
### 理论知识:
OPF = 优化发电计划（发电费用最小） + 无功电源的最优调度（网损最低）；

目标函数：总发电成本最小

$\min\Sigma_i {C_i P_i}$

约束包括：
- 功率平衡（节点功率方程）；
- 发电机出力上下限；
- 电压幅值范围。

### 代码框架：

In [ ]:
import gurobipy as gp


n_gen = 2  # 设置发电机数量
pmin = [0, 0]  #每台发电机的最小出力（单位均为MW），第0台和第1台发电机都可以从0开始发电
pmax = [6, 4]  # 每台发电机的最大出力
cost = [0.015, 0.03]  # 每台发电机的单位发电成本（单位：$/kWh），这是线性成本函数中的系数，即每发一度电的成本


load = 8  #设置系统总负荷需求，总发电量必须满足这个需求

# 创建 Gurobi 优化模型，名为opf
model = gp.Model("opf")

# 创建变量 Pg[i] 表示第 i 台发电机的出力
Pg = model.addVars(n_gen, lb=pmin, ub=pmax, name="Pg")

# 设置目标函数：总发电成本最小化，使用 gp.quicksum 表示线性加权求和
model.setObjective(
    gp.quicksum(cost[i] * Pg[i] for i in range(n_gen)),
    gp.GRB.MINIMIZE  # 指定优化目标是“最小化”
)

# 添加功率平衡约束：所有发电机出力之和必须等于负荷：Pg[0] + Pg[1] = 8
model.addConstr(
    gp.quicksum(Pg[i] for i in range(n_gen)) == load,
    name="PowerBalance"
)


model.optimize() # 启动求解器

# 打印求解结果，即每台发电机的出力
for i in range(n_gen):
    print(f"Pg[{i}] = {Pg[i].X:.2f} MW")  # 用.X 获取变量的最优值

# 打印最终的最优目标函数值，即最小发电总成本
print(f"Optimal cost: {model.ObjVal:.4f} $/h")  # 用 .ObjVal 获取目标函数值


Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 9 7945HX with Radeon Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 16 physical cores, 32 logical processors, using up to 32 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0xdfcb56b7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 3e-02]
  Bounds range     [4e+00, 6e+00]
  RHS range        [8e+00, 8e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5000000e-01   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.500000000e-01
Pg[0] = 6.00 MW
Pg[1] = 2.00 MW
Optimal cost: 0.1500 $/h


### 总结：本周任务从 Ybus 构建到 OPF 求解，涵盖建模、计算与优化三个层次。每个模块均实现对应函数，便于组合或重构为完整项目框架。